<a href="https://colab.research.google.com/github/lapatradaa/M-MMT4NL/blob/main/string_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install git+https://github.com/luozhouyang/python-string-similarity.git
!pip install pandas openpyxl


  Cloning https://github.com/luozhouyang/python-string-similarity.git to /tmp/pip-req-build-13496g2_
  Running command git clone --filter=blob:none --quiet https://github.com/luozhouyang/python-string-similarity.git /tmp/pip-req-build-13496g2_
  Resolved https://github.com/luozhouyang/python-string-similarity.git to commit 115acaacf926b41a15664bd34e763d074682bda3
  Preparing metadata (setup.py) ... done
  Created wheel for strsimpy: filename=strsimpy-0.2.1-py3-none-any.whl size=45864 sha256=9f737e4d9f44e71be7658443e9027432676b5dd87dbd655ac8c1c395ef44dda2
  Stored in directory: /tmp/pip-ephem-wheel-cache-s2x0jx46/wheels/09/83/f4/2dceea7a0542841bc4e14bcbb7259badd5031b13fdef0273f3
Successfully built strsimpy


In [4]:
!pip install -q strsimpy pandas openpyxl
!pip install -q textdistance pandas openpyxl


In [5]:
!pip install -U strsimpy
!pip install matplotlib-venn
!apt-get -qq install -y libfluidsynth1

E: Package 'libfluidsynth1' has no installation candidate


In [1]:
from google.colab import files
import pandas as pd

uploaded = files.upload()
filename = next(iter(uploaded))  # first (and only) uploaded file

df = pd.read_excel(filename) if filename.endswith((".xlsx", ".xls")) else pd.read_csv(filename)

df["references"] = df["references"].astype(str).fillna("")
df["candidates"] = df["candidates"].astype(str).fillna("")

print("file loaded:", filename)
df.head()


Saving MMT4NL_result - LLMs evaluation.csv to MMT4NL_result - LLMs evaluation.csv
file loaded: MMT4NL_result - LLMs evaluation.csv


,references,candidates,BLEU,BERT(F1)
0,ฉันมีความสุขมากที่ได้พบเพื่อนใหม่,ฉันมีความสุขมากที่ได้พบเพื่อนใหม่,100.000,1.000
1,วันนี้อากาศดีและสดชื่น,อากาศวันนี้สดชื่นและน่ารื่นรมย์,15.107,0.956
2,หล่อนรู้สึกเสียใจที่ไม่สามารถไปงานได้,เธอเสียใจที่ไม่สามารถไปร่วมงานได้,49.626,0.959
3,อาหารมื้อนี้อร่อยสุดๆ,อาหารมื้อนี้อร่อยอย่างยิ่ง,66.874,0.981
4,ฉันไม่พอใจกับบริการที่ได้รับ,ฉันไม่พอใจกับบริการที่ได้รับ,100.000,1.000


In [6]:
# ---- strsimpy module-per-class imports (like the docs screenshot) ----
from strsimpy.levenshtein import Levenshtein
from strsimpy.normalized_levenshtein import NormalizedLevenshtein
from strsimpy.jaro_winkler import JaroWinkler
from strsimpy.longest_common_subsequence import LongestCommonSubsequence
from strsimpy.sorensen_dice import SorensenDice
from strsimpy.cosine import Cosine
from strsimpy.qgram import QGram

# Your exact prompts (kept here so you can reuse in your translation pipeline)
SYSTEM_PROMPT = (
    "You are a professional translator and language expert. "
    "Please translate the following English sentence into Thai. "
    "Use English at a C1 proficiency level when framing any instructions or explanations—and output only the final Thai translation."
)
USER_PROMPT_TEMPLATE = (
    "Given this English text, translate it into Thai. Preserve the semantic of the original English text in the translate version as much as possible.\n\n"
    "{English text}"
)
def make_user_prompt(english_text: str) -> str:
    return USER_PROMPT_TEMPLATE.replace("{English text}", english_text)

# Instantiate metrics
metrics = {
    # Levenshtein (raw) isn’t normalized → we’ll normalize by max length
    "Levenshtein": Levenshtein(),
    # Normalized Levenshtein already provides normalized_* methods
    "NormalizedLevenshtein": NormalizedLevenshtein(),
    "JaroWinkler": JaroWinkler(),
    "LongestCommonSubsequence": LongestCommonSubsequence(),
    "SorensenDice": SorensenDice(),
    "Cosine(q=2)": Cosine(2),   # character bigrams
    "QGram(q=2)": QGram(2),
}
list(metrics.keys())


['Levenshtein',
 'NormalizedLevenshtein',
 'JaroWinkler',
 'LongestCommonSubsequence',
 'SorensenDice',
 'Cosine(q=2)',
 'QGram(q=2)']

In [7]:
def norm_scores(metric, a: str, b: str):
    """
    Return (normalized_similarity, normalized_distance) for any strsimpy metric.
    Fallbacks:
      - prefer normalized_* if available
      - else if similarity() exists (0..1), use 1-sim for distance
      - else if it's Levenshtein (raw), normalize by max(len(a),len(b))
    """
    # normalized_* available?
    if hasattr(metric, "normalized_similarity") and hasattr(metric, "normalized_distance"):
        sim = metric.normalized_similarity(a, b)
        dist = metric.normalized_distance(a, b)
        return sim, dist

    # generic similarity() in [0,1]?
    if hasattr(metric, "similarity"):
        sim = metric.similarity(a, b)
        dist = 1.0 - sim
        return sim, dist

    # raw Levenshtein distance (not normalized)
    if isinstance(metric, Levenshtein):
        maxlen = max(len(a), len(b))
        raw = metric.distance(a, b)
        dist = (raw / maxlen) if maxlen else 0.0
        sim = 1.0 - dist
        return sim, dist

    # last resort: try distance() and normalize by maxlen if sensible
    if hasattr(metric, "distance"):
        maxlen = max(len(a), len(b))
        raw = metric.distance(a, b)
        if maxlen:
            dist = min(1.0, max(0.0, raw / maxlen))
            sim = 1.0 - dist
            return sim, dist
        else:
            return 1.0, 0.0

    raise RuntimeError(f"Cannot score with metric {metric}")


In [8]:
REF, HYP = "references", "candidates"

for name, m in metrics.items():
    sim_col  = f"{name}_sim"
    dist_col = f"{name}_dist"
    df[sim_col], df[dist_col] = zip(*df.apply(lambda r: norm_scores(m, r[REF], r[HYP]), axis=1))

sim_cols  = sorted([c for c in df.columns if c.endswith("_sim")])
dist_cols = sorted([c for c in df.columns if c.endswith("_dist")])
df[[REF, HYP] + sim_cols + dist_cols].head(10)


✅ Calculated normalized similarity & distance.


,references,candidates,Cosine(q=2)_sim,JaroWinkler_sim,Levenshtein_sim,LongestCommonSubsequence_sim,NormalizedLevenshtein_sim,QGram(q=2)_sim,SorensenDice_sim,Cosine(q=2)_dist,JaroWinkler_dist,Levenshtein_dist,LongestCommonSubsequence_dist,NormalizedLevenshtein_dist,QGram(q=2)_dist,SorensenDice_dist
0,ฉันมีความสุขมากที่ได้พบเพื่อนใหม่,ฉันมีความสุขมากที่ได้พบเพื่อนใหม่,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,วันนี้อากาศดีและสดชื่น,อากาศวันนี้สดชื่นและน่ารื่นรมย์,0.673633,0.684751,0.354839,0.129032,0.354839,0.387097,0.500000,0.326367,0.315249,0.645161,0.870968,0.645161,0.612903,0.500000
2,หล่อนรู้สึกเสียใจที่ไม่สามารถไปงานได้,เธอเสียใจที่ไม่สามารถไปร่วมงานได้,0.707107,0.772581,0.621622,0.567568,0.621622,0.459459,0.666667,0.292893,0.227419,0.378378,0.432432,0.378378,0.540541,0.333333
3,อาหารมื้อนี้อร่อยสุดๆ,อาหารมื้อนี้อร่อยอย่างยิ่ง,0.752217,0.938081,0.653846,0.500000,0.653846,0.500000,0.697674,0.247783,0.061919,0.346154,0.500000,0.346154,0.500000,0.302326
4,ฉันไม่พอใจกับบริการที่ได้รับ,ฉันไม่พอใจกับบริการที่ได้รับ,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,หนังเรื่องนี้น่าเบื่อและยืดเยื้อ,หนังเรื่องนี้ไม่สนุกและยาวเกินไป,0.501958,0.826234,0.531250,0.187500,0.531250,0.000000,0.440678,0.498042,0.173766,0.468750,0.812500,0.468750,1.000000,0.559322
6,เขาตื่นเต้นกับโอกาสครั้งนี้,เขาตื่นเต้นกับโอกาสนี้มาก,0.800641,0.965369,0.740741,0.703704,0.740741,0.629630,0.750000,0.199359,0.034631,0.259259,0.296296,0.259259,0.370370,0.250000
7,เขาทำงานหนักและได้ผลลัพธ์ที่ดี,เขาทำงานอย่างขยันขันแข็งและได้ผลลัพธ์ที่ดี,0.679639,0.814966,0.666667,0.619048,0.666667,0.476190,0.647059,0.320361,0.185034,0.333333,0.380952,0.333333,0.523810,0.352941
8,ฉันกลัวว่าจะทำผิดพลาด,ฉันกลัวที่จะทำผิดพลาด,0.800000,0.934280,0.857143,0.809524,0.857143,0.619048,0.736842,0.200000,0.065720,0.142857,0.190476,0.142857,0.380952,0.263158
9,การเดินทางครั้งนี้สนุกและน่าจดจำ,ทริปนี้สนุกและน่าจดจำ,0.642575,0.709127,0.593750,0.531250,0.593750,0.406250,0.612245,0.357425,0.290873,0.406250,0.468750,0.406250,0.593750,0.387755


In [23]:
out_name = "string_metrics_results.xlsx"
df.to_excel(out_name, index=False)

from google.colab import files
files.download(out_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Saved: string_metrics_results.xlsx
